In [1]:
from ultralytics import YOLO
import cv2
from deepface.extendedmodels import Emotion
import numpy as np
from deepface import DeepFace
import os

# Load Sentiment analysis model

In [2]:
senti_model = Emotion.loadModel()

In [3]:
labels = {0: "angry", 1: "disgust", 2:"fear", 3:"happy", 4:"sad", 5:"surprise", 6:"neutral"}

In [40]:
from random import randint
colors = dict()
for i in range(18):
    colors["id " + str(i)] = (randint(0,255), randint(0,255), randint(0,255))
colors["No ID"] = (randint(0,255), randint(0,255), randint(0,255))

In [41]:
def sentiment_analysis(cropped_image):    
    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    cropped_image = cv2.resize(cropped_image, (48,48))
    result = senti_model.predict(np.expand_dims(cropped_image, axis=0), verbose=False)
    emotion = np.argmax(result)
    return emotion

def face_verification(cropped_image):
    results = DeepFace.find(cropped_image, db_path="database/",  model_name="OpenFace", enforce_detection=False, detector_backend = "skip", silent=True)
    if len(results[0]["identity"]) != 0:
        id = results[0]["identity"][0].split("/")[-2]
    else:
        id = "No ID"
    return id

# Write utility functions 

In [42]:
def draw(image, bounding_boxes, ids=1):
    for x, y,w,h in bounding_boxes:

        # Draw a rectangle on the image
        x = int(x) 
        y = int(y) 
        w = int(w) 
        h = int(h)

        l = int((x - w / 2))
        r = int((x + w / 2))
        t = int((y - h / 2))
        b = int((y + h / 2))
        
        cropped_image = image[t:b, l:r]
        emotion = sentiment_analysis(cropped_image)
        id = face_verification(cropped_image)
        image = cv2.rectangle(image, (l, t), (r, b), colors[id], 2)   
        image = cv2.putText(image, f"{id}: {labels[emotion]}", (l, t-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[id], 2)

    return image 

# Run the model

In [45]:
# Load the YOLOv8 model
model = YOLO('models/yolo_basic/yolov8n.pt')

# Open the video file
video_path = "/home/sawaiz/Documents/Lab/In Progress/saved/experimentation_videos/videoplayback.mp4"
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(os.path.join("model_output", "output_exp.mp4"), fourcc, fps, (frame_width,frame_height))  

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model(frame, classes=[0], verbose=False, conf=0.3)

        # Visualize the results on the frame
        annotated_frame = draw(results[0].orig_img, results[0].boxes.xywh)

        # Display the annotated frame
        # cv2.imshow("YOLOv8 Tracking", annotated_frame)

        #Write the annotated frame
        out.write(annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# Release the video capture object and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()